In [ ]:
# Importing the Keras libraries and packages
from tensorflow.keras.models import Sequential,load_model,model_from_json, Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
import numpy as np
from PIL import Image
import time
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Function if we need to invert image
def process_image(img):
    return img
    #return 1-img

In [ ]:
# Initialising the parameter
H = 550
W = 425
nb_train_samples = 353 #181 #32
nb_validation_samples = 48
epochs = 25
batch_size = 8
verbose= 1
lr = 1e-4

In [ ]:

# Creating VGG architecture for base line model
from tensorflow.keras.applications.vgg16 import VGG16

#Initialising the CNN
input_layer = Input(shape=(H,W,3))
base_model = VGG16(weights='imagenet',include_top= False,input_tensor=input_layer, pooling='avg')


top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(512, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(2, activation='softmax'))

model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
print('No of layers', len(model.layers))
print(model.layers)
opt = optimizers.Adam(learning_rate=lr)

for layer in model.layers[:-6]:
    print(layer, layer.trainable)
    layer.trainable = False


In [ ]:
#Loading previous version of model instead of creating base line model
model = load_model("/home/mldev/new_models/version_3.2/electrocardiogram.h5")
print('No of layers', len(model.layers))
for layer in model.layers[0:6]:
    print(layer.trainable)
    layer.trainable = True

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range=0.1,
                                   preprocessing_function = preprocess_image)
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  preprocessing_function = preprocess_image)

X_train_path ="/mnt/rnd/offshore/page_type/train_data_working/split_balanced_v3_electrocardiogram/train/"
X_val_path = "/mnt/rnd/offshore/page_type/train_data_working/split_balanced_v3_electrocardiogram/val/"

classes = 'electrocardiogram'

training_set = train_datagen.flow_from_directory(X_train_path,
                                                target_size = (H,W),
                                                batch_size = batch_size,
                                                classes = [f'non-{classes}',classes],
                                                class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(X_val_path,
                                                target_size = (H,W),
                                                batch_size = batch_size,
                                                classes = [f'non-{classes}',classes],
                                                class_mode = 'categorical')

#checkpoint_filepath = '/home/mldev/train_data_working/model_hw/checkpoints/vgg16_adam/weights.{epoch:03d}-{val_loss:.5f}.h5'
checkpoint_filepath = '/mnt/rnd/offshore/page_type/train_data_working/model_electrocardiogram/checkpoints_temp/vgg16_v4.0_weights.{epoch:03d}-{val_loss:.5f}.h5'
checkpointer = ModelCheckpoint(filepath=checkpoint_filepath, verbose=verbose, save_best_only=False, save_weights_only=False)

In [ ]:
model_ = model.fit(training_set,
              steps_per_epoch = nb_train_samples,
              epochs= 15,
              validation_data = test_set,
              validation_steps= nb_validation_samples,callbacks=[checkpointer])


In [ ]:

import pandas as pd

df = pd.DataFrame(model_.history)
df.index += 1
df.plot(figsize=(16,6), xticks=range(1, epochs+1),grid=True)


In [ ]:

# Prediction function

def electrocardiogram_prediction(model,img):
    img = img/255
#   img = invert_image(img)
    im=np.expand_dims(img,axis=0)
    p=model.predict(im)
    return p


In [ ]:


  # Creating list to append actual output and prediction output
label_list = []
pred_label_list = []
incorrect_preds = []
indices = {0: 'non-electrocardiogram', 1: 'electrocardiogram'}

#Test data location
og_test ='/mnt/rnd/offshore/page_type/train_data_working/split_balanced_v3_electrocardiogram/test/*'


In [ ]:

# Load model
model = load_model("/mnt/rnd/offshore/page_type/train_data_working/model_electrocardiogram/checkpoints_temp/vgg16_v4.0_weights.015-0.14895.h5")

# Running prediction for given images in for loop
for file_path in glob.glob(og_test):
    l=file_path.split('/')[len(file_path.split('/'))-1]
    for ff in glob.glob(file_path+'/*')[:500]:
        name = ff.split('/')[len(ff.split('/'))-1]
        img = cv2.imread(ff)
        pred = electrocardiogram_prediction(model,img)
        pred_label = indices[pred.argmax()]

        label_list.append(l)
        pred_label_list.append(pred_label)
        if l != pred_label:
            incorrect_preds.append(ff)
            print(f'File name: {ff}, Score: {pred}')
            print(f'Prediction value: {pred_label}, Truth value: {l}\n\n\n')


In [ ]:

#Classification report
cr = classification_report(label_list, pred_label_list, ['non-electrocardiogram', 'electrocardiogram'])
print(cr)
#TP/TN/FP/FN values
tn, fp, fn, tp = confusion_matrix(label_list, pred_label_list,labels=['non-electrocardiogram', 'electrocardiogram']).ravel()
(tn, fp, fn, tp)